In [ ]:
!pip install torchmetrics -q

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torchmetrics import Accuracy

# Model

In [ ]:
BACKBONE = 'vgg19'
device = torch.device("cuda")
model = timm.create_model(BACKBONE, num_classes=1, pretrained=True)
model = model.to(device)

# -

In [ ]:
import pandas as pd
import numpy as np
import gc
import copy
from collections import defaultdict

df = pd.read_csv("/kaggle/input/gitractcsv/data.csv")
df.head()

In [ ]:
df["empty"] = df["empty"].astype(np.int32)
df.head()

In [ ]:
from PIL import Image

def load_img(path):
    img = Image.open(path)
    img = np.expand_dims(np.array(img), axis=-1).astype('float32')
    img = np.repeat(img, 3, axis=-1)/255
    return img

In [ ]:
class UWMGIDataset(torch.utils.data.Dataset):
    def __init__(self, df, label=True, transforms=None):
        self.df = df
        self.label = label
        self.transforms = transforms
        
    def __getitem__(self, i):
        row = df.loc[i]
        img = load_img(row.image_path)
        label = row["empty"]
        
        if self.label:
            if self.transforms:
                data = self.transforms(image=img)
                img = data['image']
            img = np.transpose(img, (-1, 0, 1))
            return torch.tensor(img), torch.tensor(label)
        else:
            if self.transforms:
                data = self.transforms(image=img)
                img = data['image']
            img = np.transpose(img, (-1, 0, 1))
            return torch.tensor(img)
        
    def __len__(self):
        return len(self.df)

In [ ]:
import albumentations as A
import cv2

IMG_SIZE = [320, 384]

data_transforms = {
    "train": None,
    "valid": None
}

In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
BATCH_SIZE = 32

train_df, valid_df = train_test_split(df, test_size=.2, random_state=42)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

train_dataset = UWMGIDataset(train_df, label=True, transforms=data_transforms["train"])
valid_dataset = UWMGIDataset(valid_df, label=True, transforms=data_transforms["valid"])

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, BATCH_SIZE*2)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

def criterion(y_pred, y_true):
    return loss_fn(y_pred, y_true)

accuracy = Accuracy(task="binary").to(device)

In [ ]:
N_ACCUMULATIONS = 1
from tqdm import tqdm
from torch.cuda import amp

def train(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train')
    for step, (images, labels) in pbar:         
        images = images.to(device, dtype=torch.float)
        labels  = labels.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred.squeeze(), labels)
            loss   = loss / N_ACCUMULATIONS
            
        scaler.scale(loss).backward()
    
        if (step + 1) % N_ACCUMULATIONS == 0:
            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def valid(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid')
    for step, (images, labels) in pbar:        
        images  = images.to(device, dtype=torch.float)
        labels   = labels.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        loss    = criterion(y_pred.squeeze(), labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = nn.Sigmoid()(y_pred)
        accuracy_value  = accuracy(y_pred.squeeze(), labels).item()
        val_scores.append(accuracy_value)
    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores

In [ ]:
def full_training(model, optimizer, scheduler, device, num_epochs):
    
    
    best_model_wts = None
    best_accuracy      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_dataloader, 
                                           device=device, epoch=epoch)
        
        val_loss, val_scores = valid_one_epoch(model, valid_dataloader, 
                                                 device=device, 
                                                 epoch=epoch)
        val_accuracy = val_scores
        
        if val_accuracy >= best_accuracy:
            best_accuracy  = val_accuracy
            best_epoch   = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
    
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
from torch.optim import lr_scheduler
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-6)
EPOCHS = 5

In [ ]:
for fold in range(1):
full_training(model, optimizer, scheduler=None, device=device, num_epochs=EPOCHS)